In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import datetime
import numpy as np

In [2]:
data = pd.read_csv("frm_locations.csv")

In [3]:
data.rename(columns={'date':'date_time'},inplace=True)
data

,frm_id,date_time,latitude,longitude
0,753aa4603d9f9f250ffe7f63e77bac1c,2020-07-22T12:48:34+05:30,19.064700,72.892420
1,b96dc9bb896edad075da09f0c4e9098c,2020-07-20T09:51:41+05:30,28.460339,77.016685
2,84a9427621fe0669fbacbe6d7dc25b3d,2020-07-23T03:02:22+05:30,19.181723,72.999130
3,b96dc9bb896edad075da09f0c4e9098c,2020-07-21T21:19:46+05:30,28.452883,77.076515
4,b96dc9bb896edad075da09f0c4e9098c,2020-07-22T22:49:44+05:30,28.460367,77.016685
...,...,...,...,...
3316,f9a1994416db0f2255ef187f87c38638,2020-07-23T18:27:55+05:30,16.542560,80.578800
3317,e89712a6621c47b9485e0b06841f809f,2020-07-24T15:33:17+05:30,19.163790,73.245390
3318,84a9427621fe0669fbacbe6d7dc25b3d,2020-07-22T00:47:37+05:30,19.181705,72.999130
3319,e89712a6621c47b9485e0b06841f809f,2020-07-20T00:07:34+05:30,19.154966,73.233400


In [4]:
data['date_time'] = pd.to_datetime(data['date_time'])
data['date'] = data['date_time'].dt.date
data['time'] = data['date_time'].dt.time
data2 = data.sort_values(by=['frm_id','date','time'])
data2 = data2.drop(['date_time'],axis=1)
data2.index = pd.MultiIndex.from_frame(data2[['frm_id', 'date']])
data2 = data2.drop(['frm_id', 'date'],axis=1)
data2

latitude  longitude      time
frm_id                           date                                      
753aa4603d9f9f250ffe7f63e77bac1c 2020-07-20  18.964563  72.844330  00:24:31
                                 2020-07-20  19.064554  72.892650  00:24:36
                                 2020-07-20  19.064680  72.892395  00:39:36
                                 2020-07-20  19.064680  72.892395  00:54:33
                                 2020-07-20  19.064680  72.892395  01:09:36
...                                                ...        ...       ...
f9a1994416db0f2255ef187f87c38638 2020-07-24  16.542522  80.578850  22:34:07
                                 2020-07-24  16.542522  80.578860  22:49:06
                                 2020-07-24  16.542519  80.578865  23:08:33
                                 2020-07-24  16.542520  80.578850  23:32:51
                                 2020-07-24  16.542522  80.578850  23:48:33

[3321 rows x 3 columns]

In [5]:
user_group = data2.groupby(by=["frm_id","date"])

In [92]:
def distance(single_group):
#     print(single_group['latitude'].head(-1))
#     print(single_group['latitude'].tail(-1))
    latitude = single_group['latitude'].head(-1).subtract(single_group['latitude'].tail(-1)).abs()
    longitude = single_group['longitude'].head(-1).subtract(single_group['longitude'].tail(-1)).abs()
    dist = latitude.pow(2).add(longitude.pow(2)).pow(0.5).sum()*110
    single_group = single_group.drop(['latitude', 'longitude','time'],axis=1)
    single_group['distance'] = [dist]*len(single_group)
    return single_group

In [93]:
new_form = user_group.apply(distance).reset_index()
new_form = new_form.drop_duplicates()
new_form.index = pd.MultiIndex.from_frame(new_form[['frm_id', 'date']])
new_form = new_form['distance'].unstack(1)
new_form

date,2020-07-20,2020-07-21,2020-07-22,2020-07-23,2020-07-24
frm_id,,,,,
753aa4603d9f9f250ffe7f63e77bac1c,128.032009,0.871171,0.536755,0.392731,0.450548
84a9427621fe0669fbacbe6d7dc25b3d,0.681741,136.424678,14.390632,0.636445,0.310755
ab472ecc61608f512e4b1d4c2b49e8f8,7.007431,8.198638,35.512107,73.543000,41.847479
b96dc9bb896edad075da09f0c4e9098c,113.008037,67.148190,11996.556081,36.024442,17969.128263
e89712a6621c47b9485e0b06841f809f,91.347717,441.803525,127.479457,124.695718,258.598669
f9a1994416db0f2255ef187f87c38638,0.044105,0.291753,1012.498635,0.559266,0.567704
